In [1]:
import os
from dotenv import load_dotenv
from autogen import UserProxyAgent, ConversableAgent, AssistantAgent, GroupChat, GroupChatManager

load_dotenv()


llm_config={"model": "gpt-4o-mini"}

In [ ]:
#customer talk to project manager
customer_message = """
Hello, project manager I want to create a web-based mobile app for my insurance company where my customers can input their budget about the insurance plan, 
the reason why they need the insurance, and their status now (e.g., student, adult). 
After the app receives all the information, we can use it to recommend the best insurance plan and explain why we made that recommendation.
Please tell your team and give me back the estimate plan for this app
"""

#Project manager talk to engineers
project_manager_message = """
I have received the following message from a customer 
“ I want to create a web-based mobile app for our insurance company where my customers can input their budget about the insurance plan, 
the reason why they need the insurance, their status now (EX student, adult). 
After the app got all the information, we can base on our customer need and provide the best insurance plan for them, 
and also provide the information about why we recommend this insurance plan.”, 
Now I will talk to each engineers and please everyone give me your estimate plan about this project.

Requirement engineer please follow my work flow (Write Requirement -> Review Requirements -> Rework) and quality record (HW1, HW2) format I provide here 
give me the estimates Requirement task plan for the current project.

Quality record for Requirement task
HW1
                                  Amount of work      Productivity Rate
Write requirements                156 req             5 req/hour
Review requirement            
    Preparation for review        156 req             8 req/hour
    Review Meeting                156 req             8 req/hour
Rework                            97 defects          5 defects/hour

HW2
                                  Amount of work      Productivity Rate
Write requirements                165 req             4 req/hour
Review requirement            
    Preparation for review        165 req             5 req/hour
    Review Meeting                165 req             10 req/hour
Rework                            210 defects         8 defects/hour


///////////////////////////////////////////////////////////////////////

System engineer please follow my work flow (Write DD -> Review DD-> Rework) and quality record (HW1, HW2) format I provide here 
give me the estimates Design task plan for the current project.

Quality record for Design task 
HW1
                                  Amount of work      Productivity Rate
Write Design Document (DD)        134 pages           5 pages/hour
Review DD            
    Preparation for DD            134 req             3 pages/hour
    Review DD                     134 req             6 pages/hour
Rework                            88 defects          5 defects/hour

HW2
                                  Amount of work      Productivity Rate
Write Design Document (DD)        10 pages            6 pages/hour
Review DD            
    Preparation for DD            10 req              5 pages/hour
    Review DD                     10 req              10 pages/hour
Rework                            278 defects         10 defects/hour


///////////////////////////////////////////////////////////////////////


software engineer please follow my work flow (Write Code -> Unit Testing -> Code Inspection -> Rework) and quality record (HW1, HW2) format I provide here 
give me the estimates programming task plan for the current project.

Quality record Programming task
HW1
                                  Amount of work      Productivity Rate
Write code                        3945 SLOC           5 SLOC/hour
Unit Testing
    Prepare/Execute Test Cases    187 test case       3 test case/hour
    Fix Found Defects             155 defects         5 defects/day
    Test Fixed Defects            155 defects         10 defects/day
Code Inspection          
    Preparation for review        3945 SLOC           125 SLOC/hour
    Review Meeting                3945 SLOC           200 SLOC/hour
Rework                            314 defects         3 defects/hour

HW2
                                  Amount of work      Productivity Rate
Write code                        3450 SLOC           10 SLOC/hour
Unit Testing
    Prepare/Execute Test Cases    174 test case       5 test case/hour
    Fix Found Defects             104 defects         4 defects/day
    Test Fixed Defects            104 defects         5 defects/day
Code Inspection          
    Preparation for review        3450 SLOC           110 SLOC/hour
    Review Meeting                3450 SLOC           160 SLOC/hour
Rework                            309 defects         5 defects/hour

///////////////////////////////////////////////////////////////////////

testing engineer please follow my work flow (Write test plan(TP) -> Review TP -> Execute TP(test case) -> Fix Found Defects) 
and quality record (HW1, HW2) format I provide here 
give me the estimates Testing task plan for the current project.

Quality record for Testing task
HW1
                                  Amount of work      Productivity Rate
Write test plan (TP)              203 pages           6 pages/day
Review TP
    Preparation for TP            203 pages           2 pages/hour
    Review TP Meeting             203 pages           6 pages/hour
    Rework                        102 defects         5 defects/day
Execute TP (test cases)           310 test case       6 test case/day
Fix Found Defects                 185 defects         12 defects/day

HW2
                                  Amount of work      Productivity Rate
Write test plan (TP)              149 pages           5 pages/day
Review TP
    Preparation for TP            149 pages           5 pages/hour
    Review TP Meeting             149 pages           6 pages/hour
    Rework                        99 defects          4 defects/day
Execute TP (test cases)           138 test case       30 test case/day
Fix Found Defects                 77 defects          4 defects/day

///////////////////////////////////////////////////////////////////////

documentation engineer please follow my work flow (User Documentation (UD) -> Review UD -> Rework) and quality record (HW1, HW2) format I provide here 
give me the estimates Documentation task plan for the current project.

Quality record Documentation task
HW1
                                  Amount of work      Productivity Rate
User Documentation                141 pages           5 pages/hour
Review UD           
    Preparation for review        141 pages           4 pages/hour
    Review Meeting                141 pages           10 pages/hour
Rework                            203 defects         10 defects/hour

HW2
                                  Amount of work      Productivity Rate
User Documentation                195 pages           4 pages/hour
Review UD           
    Preparation for review        195 pages           5 pages/hour
    Review Meeting                195 pages           8 pages/hour
Rework                            344 defects         15 defects/hour

////////////////////////////////////////////////////////////////////////

Please every engineer only working on your part and give me your estimate.
Follow my quality record I provide with each task and do not estimate other engineer job
Requirement engineer working on estimate Requirement task plan 
System engineer working on estimate Design task plan
Software engineer working on estimate Programming task plan
Testing engineer working on estimate Testing task plan
Documentation engineer working on estimate Documentation task plan
"""

In [ ]:
# Agent definition
customer = UserProxyAgent(
    name="customer",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
    code_execution_config=False,
)

project_manager = AssistantAgent(
    name="project_manager",
    llm_config=llm_config,
    system_message=(
        "You are a project manager. Your task is to receive the customer's request, "
        "and send the message to your teammates. "
        "You do NOT need to analyze or expand it yourself. Focus on delivering the message for further planning."
    )
)

requirement_engineer = AssistantAgent(
    name="requirement_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a requirement engineer. You ONLY work on REQUIREMENTS tasks. "
        "Based on the insurance recommendation app project, estimate the requirements work needed. "
        "Provide your response in this EXACT format: "
        "**Requirement Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write requirements          | [your estimate]     | [your rate]         | "
        "| Review requirements         |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
        "Consider: user input forms, recommendation logic requirements, explanation features, mobile responsiveness requirements. "
        "Use realistic estimates based on project complexity."
    )
)

system_engineer = AssistantAgent(
    name="system_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a system engineer. You ONLY work on DESIGN DOCUMENT tasks. "
        "Based on the insurance recommendation app project, estimate the design documentation work needed. "
        "Provide your response in this EXACT format: "
        "**System Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write Design Document (DD)  | [your estimate]     | [your rate]         | "
        "| Review DD                   |                     |                     | "
        "|   Preparation for DD        | [your estimate]     | [your rate]         | "
        "|   Review DD                 | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
        "Consider: system architecture, database design, API design, recommendation algorithm design, security architecture. "
        "Use realistic estimates based on project complexity."
    )
)

software_engineer = AssistantAgent(
    name="software_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a software engineer. You ONLY work on PROGRAMMING/CODING tasks. "
        "Based on the insurance recommendation app project, estimate the programming work needed. "
        "Provide your response in this EXACT format: "
        "**Software Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write code                  | [your estimate]     | [your rate]         | "
        "| Unit Testing                |                     |                     | "
        "|   Prepare/Execute Test Cases| [your estimate]     | [your rate]         | "
        "|   Fix Found Defects         | [your estimate]     | [your rate]         | "
        "|   Test Fixed Defects        | [your estimate]     | [your rate]         | "
        "| Code Inspection             |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
        "Consider: frontend (forms, UI), backend (APIs, business logic), database integration, recommendation engine, mobile optimization. "
        "Use realistic estimates based on project complexity."
    )
)

testing_engineer = AssistantAgent(
    name="testing_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a testing engineer. You ONLY work on TESTING tasks. "
        "Based on the insurance recommendation app project, estimate the testing work needed. "
        "Provide your response in this EXACT format: "
        "**Testing Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| Write test plan (TP)        | [your estimate]     | [your rate]         | "
        "| Review TP                   |                     |                     | "
        "|   Preparation for TP        | [your estimate]     | [your rate]         | "
        "|   Review TP Meeting         | [your estimate]     | [your rate]         | "
        "|   Rework                    | [your estimate]     | [your rate]         | "
        "| Execute TP (test cases)     | [your estimate]     | [your rate]         | "
        "| Fix Found Defects           | [your estimate]     | [your rate]         | "
        "Consider: functional testing, UI testing, mobile testing, recommendation accuracy testing, security testing, performance testing. "
        "Use realistic estimates based on project complexity."
    )
)

documentation_engineer = AssistantAgent(
    name="documentation_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a documentation engineer. You ONLY work on DOCUMENTATION tasks. "
        "Based on the insurance recommendation app project, estimate the documentation work needed. "
        "Provide your response in this EXACT format: "
        "**Documentation Engineer**: "
        "| Category                    | Amount of work      | Productivity Rate   | "
        "|-----------------------------|---------------------|--------------------- | "
        "| User Documentation          | [your estimate]     | [your rate]         | "
        "| Review UD                   |                     |                     | "
        "|   Preparation for review    | [your estimate]     | [your rate]         | "
        "|   Review Meeting            | [your estimate]     | [your rate]         | "
        "| Rework                      | [your estimate]     | [your rate]         | "
        "Consider: user manual, help documentation, FAQ, admin documentation, API documentation. "
        "Use realistic estimates based on project complexity."
    )
)


In [ ]:
#Group chat 
groupchat = GroupChat(
    agents=[
        customer,
        project_manager,
        requirement_engineer,
        system_engineer,
        software_engineer,
        testing_engineer,
        documentation_engineer
    ],
    messages=[],
    max_round=10,
    speaker_selection_method="round_robin"
)

manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

project_manager.initiate_chat(
    manager,
    message=project_manager_message
)
final_outputs = manager.run()
print(final_outputs)

project_manager (to chat_manager):


I have received the following message from a customer 
“ I want to create a web-based mobile app for our insurance company where my customers can input their budget about the insurance plan, 
the reason why they need the insurance, their status now (EX student, adult). 
After the app got all the information, we can base on our customer need and provide the best insurance plan for them, 
and also provide the information about why we recommend this insurance plan.”, 
Now I will talk to each engineers and please everyone give me your estimate plan about this project.

Requirement engineer please follow my work flow (Write Requirement -> Review Requirements -> Rework) and quality record (HW1, HW2) format I provide here 
give me the estimates Requirement task plan for the current project.

Quality record for Requirement task
HW1
                                  Amount of work      Productivity Rate
Write requirements                156 req             5